In [1]:
import pandas as pd 
import numpy as np
import os 

In [2]:
# analysis folder version  
analysis_version = '008'

# metric version - outputs folder version 
version = '007' 

# Function 

In [3]:
def calc_stride_time_diff(stride_time_path, filename): 
    df = pd.read_csv(stride_time_path, index_col = 0) 
    df = df.reset_index(drop = True)

    # number of walks 
    num_walks = len(df)

    if num_walks > 1: 
        # calculate diff between first and last walk 
        first_walk_median_stride_time = df['stride_time_all_strides_median_sec'].iloc[0]
        last_walk_median_stride_time = df['stride_time_all_strides_median_sec'].iloc[-1]

        median_stride_time_diff = last_walk_median_stride_time - first_walk_median_stride_time

        stride_time_df = pd.DataFrame(data = {'filename': [filename], 
                                              'stride_time_num_walks' : [num_walks], 
                                              'median_stride_time_diff' : [median_stride_time_diff]}) 
    else: 
        stride_time_df = pd.DataFrame(data = {'filename': [filename], 
                                              'stride_time_num_walks' : [num_walks], 
                                              'median_stride_time_diff' : [np.nan]}) 

    return stride_time_df

In [4]:
def calc_cadence_diff(cadence_path, filename): 
    df = pd.read_csv(cadence_path, index_col = 0) 
    df = df.reset_index(drop = True)

    # number of walks 
    num_walks = len(df)

    if num_walks > 1: 
        # calculate diff between first and last walk 
        first_walk_cadence = df['cadence_step_per_min'].iloc[0]
        last_walk_cadence = df['cadence_step_per_min'].iloc[-1]

        cadence_diff = last_walk_cadence - first_walk_cadence

        cadence_df = pd.DataFrame(data = {'filename': [filename], 
                                          'cadence_num_walks' : [num_walks], 
                                          'mean_cadence_diff' : [cadence_diff]})
    # if only one walk - no first and last 
    else: 
        cadence_df = pd.DataFrame(data = {'filename': [filename], 
                                          'cadence_num_walks' : [num_walks], 
                                          'mean_cadence_diff' : [np.nan]})

    return cadence_df

In [5]:
def calc_stride_width_diff(stride_width_path, filename): 
    df = pd.read_csv(stride_width_path, index_col = 0) 
    df = df.reset_index(drop = True)
    
    # number of walks 
    num_walks = len(df)

    if num_walks > 1: 
        # calculate diff between first and last walk 
        first_walk_median_stride_width = df['stride_width_median_cm'].iloc[0]
        last_walk_median_stride_width = df['stride_width_median_cm'].iloc[-1]

        median_stride_width_diff = last_walk_median_stride_width - first_walk_median_stride_width

        stride_width_df = pd.DataFrame(data = {'filename': [filename], 
                                               'stride_width_num_walks' : [num_walks], 
                                              'median_stride_width_diff' : [median_stride_width_diff]}) 

    else: 
       stride_width_df = pd.DataFrame(data = {'filename': [filename],
                                              'stride_width_num_walks' : [num_walks], 
                                              'median_stride_width_diff' : [np.nan]})

    return stride_width_df

In [6]:
def calc_support_diff(support_path, filename): 
    df = pd.read_csv(support_path, index_col = 0) 
    df = df.reset_index(drop = True)
    
    # number of walks 
    num_walks = len(df)

    # if at least 2 walking segments and double support/single support metrics calculated 
    if (num_walks > 1) & ('tot_dsupport_per_mean' in df.columns) & ('singlesupport_per_mean' in df.columns): 
        # calculate diff between first and last walk
        # double support ---------------------------  
        first_walk_mean_dsupport_per = df['tot_dsupport_per_mean'].iloc[0]
        last_walk_mean_dsupport_per = df['tot_dsupport_per_mean'].iloc[-1]
        mean_dsupport_per_diff = last_walk_mean_dsupport_per - first_walk_mean_dsupport_per
        
        # single support -----------------------------------------------
        first_walk_mean_ssupport_per = df['singlesupport_per_mean'].iloc[0]
        last_walk_mean_ssupport_per = df['singlesupport_per_mean'].iloc[-1]
        mean_ssupport_per_diff = last_walk_mean_ssupport_per - first_walk_mean_ssupport_per

        
        support_df = pd.DataFrame(data = {'filename': [filename], 
                                               'support_num_walks' : [num_walks], 
                                               'mean_dsupport_per_diff' : [mean_dsupport_per_diff], 
                                               'mean_ssupport_per_diff' : [mean_ssupport_per_diff]}) 

    else: 
       support_df = pd.DataFrame(data = {'filename': [filename],
                                         'support_num_walks' : [num_walks], 
                                         'mean_dsupport_per_diff' : [np.nan], 
                                         'mean_ssupport_per_diff' : [np.nan]})

    return support_df

# Load data and set output folders 

In [7]:
out_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                        analysis_version, 
                        '002_video_vs_mat_metrics', 
                        'b_video_pass_by_pass', 
                        'MS_only')

if not os.path.exists(out_path): 
    os.makedirs(out_path)

In [8]:
# folder with video metrics pass by pass 
# zeno video metrics 
zv_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code',
                       'gait_bw_zeno_outputs_' + version)

print(zv_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006


# Calculate difference between first and last pass 

In [9]:
all_stride_time_diffs = [] 
all_cadence_diffs = []
all_stride_width_diffs = [] 
all_support_diffs = [] 

for (dir_path, dir_names, file_names) in os.walk(zv_path):
    
    for file_name in file_names: 
        name, ext = os.path.splitext(file_name)
        ext = ext.lower()[1:]
        current_in_path = os.path.join(dir_path, file_name) # full path to files 
        
        # calculate diff for each metric - first to last walk 
        # stride Time 
        if (ext == 'csv') & ('stride_time_stats_per_walk' in current_in_path): 
            stride_time_diff = calc_stride_time_diff(current_in_path, name)
            all_stride_time_diffs = all_stride_time_diffs + [stride_time_diff]
            
        # cadence 
        if (ext == 'csv') & ('cadence_per_walk' in current_in_path):
            cadence_diff = calc_cadence_diff(current_in_path, name)
            all_cadence_diffs = all_cadence_diffs + [cadence_diff]
            
        # stride width 
        if (ext == 'csv') & ('stride_width_stats_per_walk' in current_in_path): 
            stride_width_diff = calc_stride_width_diff(current_in_path, name)
            all_stride_width_diffs = all_stride_width_diffs + [stride_width_diff] 

        # support - use support v2, 
        if (ext == 'csv') & ('support_metrics_stats_per_walk' in current_in_path) & ('support_v2' in current_in_path): 
            support_diff = calc_support_diff(current_in_path, name) 
            all_support_diffs = all_support_diffs + [support_diff]
            
        

In [10]:
# create data frames and save 
all_stride_time_diffs_df = pd.concat(all_stride_time_diffs, ignore_index = True)
all_stride_time_diffs_df.to_csv(os.path.join(out_path, 'all_stride_time_diffs.csv')) 
                                
all_cadence_diffs_df = pd.concat(all_cadence_diffs, ignore_index = True)
all_cadence_diffs_df.to_csv(os.path.join(out_path, 'all_cadence_diffs.csv')) 

all_stride_width_diffs_df = pd.concat(all_stride_width_diffs, ignore_index = True)
all_stride_width_diffs_df.to_csv(os.path.join(out_path, 'all_stride_width_diffs.csv')) 

all_support_diffs_df = pd.concat(all_support_diffs, ignore_index = True) 
all_support_diffs_df.to_csv(os.path.join(out_path, 'all_support_diffs.csv')) 

In [11]:
all_support_diffs_df.head()

,filename,support_num_walks,mean_dsupport_per_diff,mean_ssupport_per_diff
0,gait_vertical_FW_1_BW-0002_2023_09_12_support_...,2,2.29,1.50
1,gait_vertical_PWS_1_BW-0002_2023_09_12_support...,2,NaN,NaN
2,gait_vertical_FW_1_BW-0003_2022_10_24_support_...,1,NaN,NaN
3,gait_vertical_PWS_1_BW-0003_2022_10_24_support...,2,-1.41,-0.91
4,gait_vertical_FW_1_BW-0004_2022_09_19_support_...,0,NaN,NaN


In [12]:
# Preferred Walking Speed 

# stride time 
pws_stride_time_diffs_df = all_stride_time_diffs_df.loc[all_stride_time_diffs_df['filename'].str.contains('PWS')]
pws_stride_time_mean_diff = round(pws_stride_time_diffs_df['median_stride_time_diff'].mean(), 2)
pws_stride_time_abs_mean_diff = round(pws_stride_time_diffs_df['median_stride_time_diff'].abs().mean(), 2)

# cadence 
pws_cadence_diffs_df = all_cadence_diffs_df.loc[all_cadence_diffs_df['filename'].str.contains('PWS')] 
pws_cadence_mean_diff = round(pws_cadence_diffs_df['mean_cadence_diff'].mean(), 2) 
pws_cadence_abs_mean_diff = round(pws_cadence_diffs_df['mean_cadence_diff'].abs().mean(), 2)

# stride width 
pws_stride_width_diffs_df = all_stride_width_diffs_df.loc[all_stride_width_diffs_df['filename'].str.contains('PWS')] 
pws_stride_width_mean_diff = round(pws_stride_width_diffs_df['median_stride_width_diff'].mean(), 2) 
pws_stride_width_abs_mean_diff = round(pws_stride_width_diffs_df['median_stride_width_diff'].abs().mean(), 2) 

# support 
pws_support_diffs_df = all_support_diffs_df.loc[all_support_diffs_df['filename'].str.contains('PWS')] 
pws_dsupport_mean_diff = round(pws_support_diffs_df['mean_dsupport_per_diff'].mean(), 2) 
pws_dsupport_abs_mean_diff = round(pws_support_diffs_df['mean_dsupport_per_diff'].abs().mean(), 2)

pws_ssupport_mean_diff = round(pws_support_diffs_df['mean_ssupport_per_diff'].mean(), 2) 
pws_ssupport_abs_mean_diff = round(pws_support_diffs_df['mean_ssupport_per_diff'].abs().mean(), 2)

# merge and save 
pws_all_mean_diffs = pd.DataFrame(data = {'median_stride_time' : [pws_stride_time_mean_diff], 
                                          'mean_cadence' : [pws_cadence_mean_diff], 
                                          'median_stride_width' : [pws_stride_width_mean_diff], 
                                          'mean_dsupport_per' : [pws_dsupport_mean_diff],
                                          'mean_ssupport_per' : [pws_ssupport_mean_diff]}) 

pws_all_mean_diffs.to_csv(os.path.join(out_path, 'pws_video_walks_mean_diff.csv')) 


pws_all_abs_mean_diffs = pd.DataFrame(data = {'median_stride_time' : [pws_stride_time_abs_mean_diff], 
                                              'mean_cadence' : [pws_cadence_abs_mean_diff], 
                                              'median_stride_width' : [pws_stride_width_abs_mean_diff],
                                              'mean_dsupport_per' : [pws_dsupport_abs_mean_diff],
                                              'mean_ssupport_per' : [pws_ssupport_abs_mean_diff]})
pws_all_abs_mean_diffs.to_csv(os.path.join(out_path, 'pws_video_walks_mean_abs_diff.csv'))

In [13]:
# Fast walking videos 
# stride time 
fw_stride_time_diffs_df = all_stride_time_diffs_df.loc[all_stride_time_diffs_df['filename'].str.contains('FW')]
fw_stride_time_mean_diff = round(fw_stride_time_diffs_df['median_stride_time_diff'].mean(), 2)
fw_stride_time_abs_mean_diff = round(fw_stride_time_diffs_df['median_stride_time_diff'].abs().mean(), 2)

# cadence 
fw_cadence_diffs_df = all_cadence_diffs_df.loc[all_cadence_diffs_df['filename'].str.contains('FW')] 
fw_cadence_mean_diff = round(fw_cadence_diffs_df['mean_cadence_diff'].mean(), 2) 
fw_cadence_abs_mean_diff = round(fw_cadence_diffs_df['mean_cadence_diff'].abs().mean(), 2)

# stride width 
fw_stride_width_diffs_df = all_stride_width_diffs_df.loc[all_stride_width_diffs_df['filename'].str.contains('FW')] 
fw_stride_width_mean_diff = round(fw_stride_width_diffs_df['median_stride_width_diff'].mean(), 2) 
fw_stride_width_abs_mean_diff = round(fw_stride_width_diffs_df['median_stride_width_diff'].abs().mean(), 2) 

# support 
fw_support_diffs_df = all_support_diffs_df.loc[all_support_diffs_df['filename'].str.contains('FW')] 
fw_dsupport_mean_diff = round(fw_support_diffs_df['mean_dsupport_per_diff'].mean(), 2) 
fw_dsupport_abs_mean_diff = round(fw_support_diffs_df['mean_dsupport_per_diff'].abs().mean(), 2)

fw_ssupport_mean_diff = round(fw_support_diffs_df['mean_ssupport_per_diff'].mean(), 2) 
fw_ssupport_abs_mean_diff = round(fw_support_diffs_df['mean_ssupport_per_diff'].abs().mean(), 2)

# merge and save 
fw_all_mean_diffs = pd.DataFrame(data = {'median_stride_time' : [fw_stride_time_mean_diff], 
                                          'mean_cadence' : [fw_cadence_mean_diff], 
                                          'median_stride_width' : [fw_stride_width_mean_diff],
                                          'mean_dsupport_per' : [fw_dsupport_mean_diff],
                                          'mean_ssupport_per' : [fw_ssupport_mean_diff]}) 

fw_all_mean_diffs.to_csv(os.path.join(out_path, 'fw_video_walks_mean_diff.csv')) 


fw_all_abs_mean_diffs = pd.DataFrame(data = {'median_stride_time' : [fw_stride_time_abs_mean_diff],
                                             'mean_cadence' : [fw_cadence_abs_mean_diff],
                                             'median_stride_width' : [fw_stride_width_abs_mean_diff],
                                             'mean_dsupport_per' : [fw_dsupport_abs_mean_diff],
                                             'mean_ssupport_per' : [fw_ssupport_abs_mean_diff]}) 
fw_all_abs_mean_diffs.to_csv(os.path.join(out_path, 'fw_video_walks_mean_abs_diff.csv'))